In [ ]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time
from math import ceil,floor

In [5]:
def g1(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]
    return 85.334407 + 0.0056858*x2*x5 + 0.0006262*x1*x4 - 0.0022053*x3*x5

In [6]:
def g2(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]
    
    return 80.51249 + 0.0071317*x2*x5 + 0.0029955*x1*x2 - 0.0021813*x3**2

In [7]:
def g3(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]

    return 9.300961 + 0.0047026*x3*x5 + 0.0012547*x1*x3 + 0.0019085*x3*x4

In [8]:
def is_feasable(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]
    if x1<78  or x1>102:
        print("x1")
        return False
    if x2<33 or x2>45:
        print("x2")
        return False
        
    if x3<27 or x3>45:
        print("x3")
        return False
    if x4<27 or x4>45:
        print("x4")
        return False
    if x5<27 or x5>45:
        print("x5")
        return False

    g1res=g1(x)
    g2res=g2(x)
    g3res=g3(x)

    if g1res<0 or g1res>92:
        print("g1")
        return False
    if g2res<90 or g2res>110:
        print("g2")
        return False
    if g3res<20 or g3res>25:
        print("g3")
        return False
    
    return True

In [9]:
def himmelblau(x):
    x3=x[2]
    x1=x[0]
    x5=x[4]
    toReturn= -(5.3578547*x3**2 + 0.8356891*x1*x5 + 37.293239*x1 - 40792.141)
    g1res=g1(x)
    if g1res>92:
        toReturn+=-((g1res)-92)*20000
    elif g1res<0:
        toReturn+=(g1res)*20000
    
    g2res=g2(x)
    if g2res>110:
        toReturn-=((g2res)-110)*20000
    elif g2res<90:
        toReturn-=(90-(g2res))*20000
    
    g3res=g3(x)
    if g3res>25:
        toReturn-=((g3res)-25)*10000
    elif g3res<20:
        toReturn-=(20-(g3res))*10000
    return toReturn

In [10]:
def actual_himmelblau(x):
    x3=x[2]
    x1=x[0]
    x5=x[4]
    return -(5.3578547*x3**2 + 0.8356891*x1*x5 + 37.293239*x1 - 40792.141)

In [11]:
class Particle:
    global_best_position = None
    global_best_value = None
    
    def __init__(self, bounds, obj_func, c_i,
                 c_p, c_g):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))

        self.personal_best_position = self.position.copy()
        
        self.obj_func = obj_func
        self.c_i = c_i
        self.c_p = c_p
        self.c_g = c_g
        
        self.value = obj_func(self.position)
        self.personal_best_value = obj_func(self.personal_best_position)

        if Particle.global_best_value is None or Particle.global_best_value > self.value:
            Particle.global_best_value = self.value
            Particle.global_best_position = self.position.copy()
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.value = self.obj_func(self.position)
        if self.value < self.personal_best_value:
            self.personal_best_value = self.value
            self.personal_best_position = self.position.copy()
            if self.value < Particle.global_best_value:
                Particle.global_best_value = self.value
                Particle.global_best_position = self.position.copy()
    
    def update_velocity(self):
        r_p = np.random.random(self.velocity.shape)
        r_s = np.random.random(self.velocity.shape)
# mozda bi imalo smisla da normiramo vektore pre ovoga
# mozda i value ukljuciti pazljivo
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Particle.global_best_position - self.position)
        
        #if any([x>10**(-8) for x in personal_direction]):
         #   personal_direction/=np.linalg.norm(personal_direction)
        #if any([x!=0 for x in global_direction]):
         #   global_direction/=np.linalg.norm(global_direction)
        
        cognitive_velocity = r_p * self.c_p * (personal_direction)#*np.linalg.norm(self.velocity)
        social_velocity = r_s * self.c_g * (global_direction)#*np.linalg.norm(self.velocity)
        inertia = self.c_i * self.velocity
        self.velocity = np.clip(inertia + cognitive_velocity + social_velocity,2*self.lower_bounds,2*self.upper_bounds)
        

In [12]:

def pso(swarm_size, num_dimensions, c_i, c_p, c_g, allowed_time,func_to_minimize):
    bounds = [(78, 102),(33,45),(27,45),(27,45),(27,45)]
    
    swarm = [Particle(bounds=bounds,
                      obj_func=func_to_minimize,
                      c_i=c_i,
                      c_p=c_p,
                      c_g=c_g) for _ in range(swarm_size)]
    #print(Particle.global_best_value)
    start=time.time()
    while True:
        for particle in swarm:
            particle.update_velocity()
            particle.update_position()
        if time.time() - start>allowed_time:
            tmp=actual_himmelblau(Particle.global_best_position)
            if  not is_feasable(Particle.global_best_position):
                print("not feasable!")
                print(g1(Particle.global_best_position))
                print(g2(Particle.global_best_position))
                print(g3(Particle.global_best_position))
            Particle.global_best_position=None
            Particle.global_best_value=None
            return tmp

In [13]:
pso(swarm_size=50, num_dimensions=2, c_i=0.7, c_p=1.0, c_g=1.0, allowed_time=0.1,func_to_minimize=rosenbrock)

g1
not feasable!
92.30118908619389
95.93395111367774
23.29307660197386


27964.523912793542

In [14]:
times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000

In [127]:
pso(swarm_size=50, num_dimensions=5, c_i=0.7, c_p=1.0, c_g=1.0, allowed_time=1,func_to_minimize=himmelblau)

g1
not feasable!
95.2566775
104.28639500000001
28.4475115


22302.761885500004

In [16]:
 print("Function : Shaffer")
print("Swarm size: " + "50")
print("Num dimensions: " + "2")
print("c_i: "+"0.7")
print("c_p: " + "1.0")
print("c_g: " + "1.0")
for allowed_time in times:
    suma=0
    for _ in range(num_trys):
        suma+=pso(swarm_size=50, num_dimensions=2, c_i=0.7, c_p=1.0, c_g=1.0, allowed_time=allowed_time,func_to_minimize=shaffer)
    suma/=num_trys
   
    print("    Time : " + str(allowed_time))
    print("    Average mistake: "+ str(suma))
    print("\n")

Function : Shaffer
Swarm size: 50
Num dimensions: 2
c_i: 0.7
c_p: 1.0
c_g: 1.0
g1
not feasable!
93.63100144804343
99.11891185974366
25.424032803267085
g1
not feasable!
92.96432686878109
97.66625987475753
25.989382764256156
g1
not feasable!
92.94954200937445
98.93238069430085
21.846901993746016
g3
not feasable!
90.91844185330338
94.8983438930275
17.781755416301475
g1
not feasable!
92.18945852800245
96.68137941353353
23.6896930097194
g1
not feasable!
93.32646987251117
99.34236716978594
21.017437717911243
g1
not feasable!
92.76071756673713
98.21564715885816
23.211146264784023
g1
not feasable!
93.26490194504626
98.15799642144391
24.718874181214897
g1
not feasable!
92.8760035334115
99.7029355299634
23.07375784166262
g1
not feasable!
92.86489561963987
98.1569827162577
22.995753980996337
g1
not feasable!
93.8236006088382
99.29481152920368
20.044808804215982
g1
not feasable!
92.43342409191659
96.7554269187366
24.819743801573864
g1
not feasable!
93.02133998421237
97.92553009186854
18.9303931145

KeyboardInterrupt: 

In [ ]:
rastrigin([1,2])